<a href="https://colab.research.google.com/github/adarsh415/gpt/blob/main/gpt_Block_FeedForward_multiHead_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-22 17:58:06--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-07-22 17:58:06 (30.6 MB/s) - ‘input.txt.4’ saved [1115394/1115394]



In [108]:
# read it to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [109]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [110]:
#let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [111]:
# here are all unique characters that occurs in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [112]:
# create mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: takes a string, returns sequence of intgers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes list of integers, outputs string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [113]:
import torch
from torch import nn
import torch.nn.functional as F

In [114]:
batch_size=32 # how many independent sequences will process in parallel
block_size=8 # what is the maximum context length of predictions?
n_embed = 32
lr_rate = 1e-3
eval_interval = 500
eval_iters = 200
max_iters = 5000

In [115]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [116]:
#Let's split dataset into train and validation set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [117]:
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [118]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when context is: {context} target is: {target}")

when context is: tensor([18]) target is: 47
when context is: tensor([18, 47]) target is: 56
when context is: tensor([18, 47, 56]) target is: 57
when context is: tensor([18, 47, 56, 57]) target is: 58
when context is: tensor([18, 47, 56, 57, 58]) target is: 1
when context is: tensor([18, 47, 56, 57, 58,  1]) target is: 15
when context is: tensor([18, 47, 56, 57, 58,  1, 15]) target is: 47
when context is: tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is: 58


In [119]:
torch.manual_seed(1023)
def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)

print('---------')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when context is: {context.tolist()} target is: {target}")


inputs
torch.Size([32, 8])
tensor([[53,  1, 40, 43,  1, 58, 39, 50],
        [ 1, 42, 53,  1, 52, 53, 58,  1],
        [47, 58,  1, 47, 57,  1, 52, 53],
        [25, 13, 30, 19, 13, 30, 17, 32],
        [43,  1, 39, 58,  1, 58, 46, 63],
        [46, 47, 57,  1, 46, 53, 50, 63],
        [ 0, 26, 53, 61,  1, 57, 61, 39],
        [ 1, 52, 39, 51, 43,  1, 40, 43],
        [ 0, 25, 17, 26, 17, 26, 21, 33],
        [13, 16, 37,  1, 19, 30, 17, 37],
        [ 1, 59, 54,  1, 58, 46, 43,  1],
        [45, 46, 52, 43, 57, 57,  6,  1],
        [53, 10,  0, 61, 46, 63,  6,  1],
        [48, 53, 47, 52,  5, 42,  0, 14],
        [42,  8,  0,  0, 19, 24, 27, 33],
        [46, 39, 58,  1, 24, 43, 61, 47],
        [43, 56, 43,  2,  0,  0, 14, 33],
        [35, 43, 50, 41, 53, 51, 43,  6],
        [26, 34, 27, 24, 21, 27, 10,  0],
        [57,  1, 40, 50, 47, 52, 42,  1],
        [58, 46, 43, 56,  1, 57, 58, 39],
        [51, 39, 50, 50,  1, 39, 50, 43],
        [43, 58, 41, 46,  1, 57, 46, 56],
       

In [120]:
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embed, head_size, bias=False)
    self.query = nn.Linear(n_embed, head_size, bias=False)
    self.value = nn.Linear(n_embed, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.query(x)
    # compute the attention scores ('affinities)
    wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) ---> (B, T, T) , used scaled attention here
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) , this is part of decoder so does not communicate with past
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    # perform the weighted aggregation of values
    v = self.value(x) # (B, T, C)
    out = wei @ v # (B, T, T) @ (B, T, C) --> (B, T, C)
    return out

In [122]:
class MultiHeadAttention(nn.Module):
  """Multi head attention"""
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.head = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

  def forward(self, x):
    return torch.cat([h(x) for h in self.head], dim=-1) # concatenate along the channel dimension, i.e last dimension

In [130]:
class FeedForward(nn.Module):
  """Simple linear layer followed by non linearilty"""

  def __init__(self, n_embed):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embed, n_embed),
        nn.ReLU()
    )
  def forward(self, x):
    return self.net(x)

In [141]:
class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  def __init__(self, n_embed, n_head):
    super().__init__()
    # n_embed: embedding dimension, n_head: the number of heads we'd like
    head_size = n_embed // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embed)

  def forward(self, x):
    x = self.sa(x)
    x = self.ffwd(x)
    return x


In [142]:
torch.manual_seed(1033)

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super(BigramLanguageModel, self).__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    self.blocks = nn.Sequential(Block(n_embed, n_head=4),
        Block(n_embed, n_head=4),
        Block(n_embed, n_head=4)
    )
    self.lm_head = nn.Linear(n_embed, vocab_size)

  def forward(self, idx, target=None):
    B, T = idx.shape

    # idx and targets are both (B,T) tensors of integer
    token_embed = self.token_embedding_table(idx) # (B,T,C) where C is n_embed
    pos_embed = self.position_embedding_table(torch.arange(T)) # (T,C)
    X = token_embed + pos_embed # (B, T, C)
    X = self.blocks(X) #(B, T, C)
    logits = self.lm_head(X) # (B, T, Vocab_size)

    if target is not None:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(-1) # or target.view(B*T)
      loss = F.cross_entropy(logits, target)

    else:
      loss = None

    return logits, loss

  def generate(self, idx, max_new_char):
    #idx is (B, T) array of indices in current context
    for _ in range(max_new_char):
      # print(idx.shape)
      # print(idx)
      # crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      # print(idx_cond)
      # get the predictions
      logits, loss = self(idx_cond)
      # Focus only on last time step
      logits = logits[:, -1, :] # becomes [B, C]
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from distribution
      idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
      # append sampled index to running index
      idx = torch.cat((idx, idx_next), dim=1) # (b , T+1)

    return idx

m = BigramLanguageModel()
logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# start_token = torch.zeros((1,1), dtype=torch.long)
# print(decode(m.generate(start_token, 100)[0].tolist()))

In [143]:
# create Optimizer
optimizer = torch.optim.Adam(m.parameters(), lr = lr_rate)

In [144]:
@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      xb, yb = get_batch(split)
      logits, loss = m(xb, yb)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [145]:
for steps in range(max_iters):

  if steps % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step 0: train loss 4.1530, val loss 4.1547
step 500: train loss 3.1124, val loss 3.1164
step 1000: train loss 2.7853, val loss 2.7867
step 1500: train loss 2.6315, val loss 2.6270
step 2000: train loss 2.5239, val loss 2.5395
step 2500: train loss 2.4592, val loss 2.4558
step 3000: train loss 2.4084, val loss 2.4031
step 3500: train loss 2.3598, val loss 2.3644
step 4000: train loss 2.3315, val loss 2.3514
step 4500: train loss 2.3026, val loss 2.3036
2.3015568256378174


In [146]:
start_token = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(start_token, 1000)[0].tolist()))


Lhat tloy py it shud at mom averld she o thy awilrs hat?
Kaye offhat youal.
IOVES:
Thoud?

Merant, sprnave sud shen demen
Bhat I twicte miwined as the yave of his, the wiseare grleit noo, mis chiveg gry yous nove sew
An the suea omwy prepove ive sume sheden becor'lt day lark make hi spe dee-Rtimee whane fims me whas o's grov thall whe then yel of preer,
Thom he our alle, neses novy nes, I me-
Whee, -my not lothe I horp ins heate wouny whe his relss are ade ex my pare the by ther at ming.
Namive nur.

CA Whouns.

MERN:
Cher alke aseam, pry bedeank I thun share thy lelat men ttoul stame fheveles toood deth this lis Bud me!
He ther?
Thats:
I vaun loldine,' le wives? fon? 
Loughsse?

Vouchne wovean,
wher of nos carls?

Moube us stile frome he'r whatt got nouv tous uw wy of gou
gol mirt rousttie; shis we aan lis thore Thy for me,
Lenoseg eoa bry lick qoomer Ey coome of ile khes yopes, whered swrerem ad hhesing I fe.
Ceven, Yeshigh'aid nomy iw we whast Intrae, dor mouve aly thires ad rew
It